In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2022/data.csv')

In [ ]:
df.isna().sum(axis=0)

In [ ]:
arr = df.fillna(0).corr()

In [ ]:
high_corr = np.argwhere(arr.values > 0.05)
high_corr = high_corr[high_corr[:,0] != high_corr[:,1],:]
high_corr

We can observe that only elements in F_2 and F_4 are correlated. F_2 is not a target so we focus on F_4.

In [ ]:
df.isna().sum(axis=1).value_counts()

There are varying number of NA entries in each row. To accomodate this, we take training/validation data from the rows with no NA entries, and develop models with one target variable (and the rest predictor variables). We impute the remaining NA values by mean when making the prediction.

In [ ]:
df_train = df[df.isna().sum(axis=1) == 0].drop('row_id', axis=1)
df_train.shape

In [ ]:
df.iloc[:,1:].std()

In [ ]:
#F_1
#df_train_f1 = df_train.iloc[:,:15]
#This gives MSE by using the mean.
#df_train_f1.var(axis=0)

In [ ]:
#from sklearn.model_selection import train_test_split
#X = df_train_f1.drop('F_1_0', axis=1)
#y = df_train_f1['F_1_0']
#X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
#from sklearn.neighbors import KNeighborsRegressor
#knn = KNeighborsRegressor()
#knn.fit(X_train, y_train)

In [ ]:
#preds = knn.predict(X_test.iloc[0:1000,:])
#print(np.mean(np.square(preds - y_test.values[:1000])))

It seems that knn is even worse than just using mean. Let's try this on other columns.

In [ ]:
#X = df_train.drop('F_1_0', axis=1)
#y = df_train['F_1_0']
#X_train, X_test, y_train, y_test = train_test_split(X, y)
#knn = KNeighborsRegressor()
#knn.fit(X_train, y_train)
#preds = knn.predict(X_test.iloc[0:1000,:])
#print(np.mean(np.square(preds - y_test.values[:1000])))

In [ ]:
def knn(col, df):
    X = df.drop(col, axis=1)
    y = df[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    knn = KNeighborsRegressor()
    knn.fit(X_train, y_train)
    preds = knn.predict(X_test.iloc[0:1000,:])
    print(np.mean(np.square(preds - y_test.values[:1000])))

In [ ]:
#for j in range(1,15):
#    col = 'F_1_'+str(j)
#    knn(col, df_train)
#    knn(col, df_train_f1)

We see that this is in general worse than using mean.

In [ ]:
#F_4
df_train_f4 = df_train.iloc[:,15+25+25:]
df_train_f4.corr()

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
def model_linear(col, df, num_round, l=0, mask=False):
    X = df.drop(col, axis=1)
    y = df[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8)
    param = {'objective': 'reg:squarederror', 'booster': 'gblinear', 'eval_metric': 'rmse', 'lambda': l}
    dtrain = xgb.DMatrix(data=X_train, label=y_train)
    dtest = xgb.DMatrix(data=X_val, label=y_val)
    evallist = [(dtest, 'eval'), (dtrain, 'train')]
    bst = xgb.train(param, dtrain, num_round, evallist, verbose_eval=False)
    #Randomly mask some values
    if mask:
        masks = np.random.choice(14, X_test.shape[0])
        temp = X_test.values
        temp[np.arange(X_test.shape[0]),masks] = 0
        preds = bst.predict(xgb.DMatrix(pd.DataFrame(temp, columns = X_test.columns)))
    else:
        preds = bst.predict(xgb.DMatrix(X_test))
    print(col, np.sqrt(mean_squared_error(y_test, preds)), df[col].std())
    #xgb.plot_importance(bst)
    return bst

In [ ]:
def model_tree(col, df, num_round, l=1, mask=False):
    X = df.drop(col, axis=1)
    y = df[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8)
    param = {'objective': 'reg:squarederror', 'booster': 'gbtree', 'eval_metric': 'rmse', 'lambda': l}
    dtrain = xgb.DMatrix(data=X_train, label=y_train)
    dtest = xgb.DMatrix(data=X_val, label=y_val)
    evallist = [(dtest, 'eval'), (dtrain, 'train')]
    bst = xgb.train(param, dtrain, num_round, evallist, verbose_eval=False)
    if mask:
        #Randomly mask some values
        masks = np.random.choice(14, X_test.shape[0])
        temp = X_test.values
        temp[np.arange(X_test.shape[0]),masks] = 0
        preds = bst.predict(xgb.DMatrix(pd.DataFrame(temp, columns = X_test.columns)))
    else:
        preds = bst.predict(xgb.DMatrix(X_test))
    print(col, np.sqrt(mean_squared_error(y_test, preds)), df[col].std())
    #xgb.plot_importance(bst)
    return bst

We first try the linear method (seeing the relatively high correlations).

In [ ]:
#for i in df_train_f4.columns:
#    model_linear(i, df_train_f4, 100)

This does less well on columns 0, 3, 6, 9. We may have to try a different method. (trees)
Also note this method does not do well if there are NA values in the features.

In [ ]:
#for i in ['F_4_0', 'F_4_3', 'F_4_6', 'F_4_9']:
#    model_tree(i, df_train_f4, 100)

This seems to do a slightly better job. We stick with these models.

In [ ]:
models = []
for i in df_train_f4.columns:
    model = []
    #Normal model
    if i in ['F_4_0', 'F_4_3', 'F_4_6', 'F_4_9']:
        model.append(model_tree(i, df_train_f4, 100, 0.5, False))
    else:
        model.append(model_linear(i, df_train_f4, 200, 0, False))
    #Model for entries with one or more NA values
    model.append(model_tree(i, df_train_f4, 50, 1, True))
    models.append(model)

In [ ]:
preds = []
for i in range(15):
    arr = df[df[df.columns[15+25+25+i+1]].isna()].iloc[:,1+15+25+25:]
    arr = arr.drop([df.columns[15+25+25+i+1]], axis=1)
    #Use linear/tree model
    normal = xgb.DMatrix(arr[arr.isna().sum(axis=1) == 0])
    more = xgb.DMatrix(arr[arr.isna().sum(axis=1) > 0])
    preds.append([models[i][0].predict(normal), models[i][1].predict(more)])

In [ ]:
for i in preds:
    print(i[0].shape, i[1].shape)

In [ ]:
sub = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2022/sample_submission.csv')
missing_cols = np.argwhere(df.isna().values)
print(missing_cols)
means = df.mean(axis=0).values
print(means)

In [ ]:
#Is there a way to make this faster?
counts = []
for i in range(15):
    counts.append([0,0])
result = []
for i in range(missing_cols.shape[0]):
    ind = missing_cols[i][1]
    if ind < 15+25+25+1:
        result.append(means[ind])
    else:
        feats = df.iloc[missing_cols[i][0],:].drop(df.columns[ind]).iloc[15+25+25+1:]
        na = 1 if feats.isna().sum() > 0 else 0
        try:
            result.append(preds[ind-15-25-25-1][na][counts[ind-15-25-25-1][na]])
        except:
            print(ind, feats, na, counts)
        counts[ind-15-25-25-1][na] += 1
sub['value'] = np.array(result)

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)